In [9]:
import cv2 as cv
import numpy as np

def read_opt(name):
    img1 = cv.imread(name)
    height = 400
    width = int(img1.shape[1] * height / img1.shape[0])
    img1 = cv.resize(img1, (width, height))
    return img1

def clahe_opt(img1):
    # 增加对比度
    img1 = cv.cvtColor(img1, cv.COLOR_BGR2LAB)
    l, a, b = cv.split(img1)
    clahe = cv.createCLAHE(clipLimit=2, tileGridSize=(8, 8))
    clahe_l = clahe.apply(l)
    img2 = cv.merge([clahe_l, a, b])
    img2 = cv.cvtColor(img2, cv.COLOR_LAB2BGR)
    return img2

def lab_opt(img1):
    # 做高通滤波模糊处理
    rows, cols, channels = img1.shape
    gaussian_sigma = 50  # 高斯滤波器的标准差
    gaussian_filter = np.zeros((rows, cols, 2 ), np.float32)
    for i in range(rows):
        for j in range(cols):
            gaussian_filter[i, j] = np.exp(-((i - rows // 2) ** 2 + (j - cols // 2) ** 2) / (2 * gaussian_sigma ** 2))

    img0 = cv.cvtColor(img1, cv.COLOR_BGR2LAB)
    img2 = img0[:, :, 1]**2
    img4 = img0[:, :, 2]**2

    dft = cv.dft(np.float32(img2),flags = cv.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    dft_filtered = dft_shift * gaussian_filter
    f_ishift = np.fft.ifftshift(dft_filtered)
    img2 = cv.idft(f_ishift)
    img2 = cv.magnitude(img2[:,:,0],img2[:,:,1])
    img2 = cv.normalize(img2, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)

    dft = cv.dft(np.float32(img4),flags = cv.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    dft_filtered = dft_shift * gaussian_filter
    f_ishift = np.fft.ifftshift(dft_filtered)
    img4 = cv.idft(f_ishift)
    img4 = cv.magnitude(img4[:,:,0],img4[:,:,1])
    img4 = cv.normalize(img4, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)

    img0[:, :, 0]  = cv.add(img2,img4)
    img0 = cv.cvtColor(img0,cv.COLOR_LAB2BGR)

    return  img0

def color_opt(img1):
    # 颜色阈值
    img0 = cv.cvtColor(img1,cv.COLOR_BGR2HSV)
    lower_red = np.array([0, 0, 96])
    upper_red = np.array([180, 255, 255])
    mask = cv.inRange(img1, lower_red, upper_red)
    # 在原始图像上应用掩膜，提取水果部分
    result = cv.bitwise_and(img1, img1, mask=mask)
    return result

def canny_opt(img1):
    # 边缘检测
    edges = cv.Canny(img1,100,200)
    return edges

def contour_opt(img1):
    # 轮廓检测
    img2 = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
    ret,img3 = cv.threshold(img2,100,255,cv.THRESH_BINARY)
    cv.imshow("contDour",img3)
    contours,hierarchy = cv.findContours(img3,cv.RETR_LIST,cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(img1, contours, -1, (0,255,0), 3)
    return img1

img = read_opt("111.jpg")
cv.imshow("img",img)
img = clahe_opt(img)
img = lab_opt(img)
cv.imshow("lab",img)
img = color_opt(img)
cv.imshow("color",img)
# img = contour_opt(img)
# cv.imshow("contour",img)

cv.waitKey(0)
cv.destroyAllWindows()